## Clustering
Put simply, the task of clustering is to place observations that seem similar within the same cluster. Clustering is commonly used in two dimensional data where the goal is to create clusters based on coordinates. Here, we will use something similar. We will cluster houses based on their latitude-longitude locations using several different clustering methods.

In [ ]:
# Packages we will use throughout this notebook
using Clustering
using VegaLite
using VegaDatasets
using DataFrames
using Statistics
using JSON
using CSV
using Distances

We will start off by getting some data. We will use data of 20,000+ California houses dataset. We will then learn whether housing prices directly correlate with map location.

In [ ]:
download("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv","newhouses.csv")
houses = CSV.read("newhouses.csv")

In [ ]:
names(houses)

We will use the `VegaLite` package here for plotting. This package makes it very easy to plot information on a map. All you need is a JSON file of the map you intend to draw. Here, we will use the California counties JSON file and plot each house on the map and color code it via a heatmap of the price. This is done by this line `color="median_house_value:q"`

In [ ]:


cali_shape = JSON.parsefile("downloads/california-counties.json")
VV = VegaDatasets.VegaJSONDataset(cali_shape,"downloads/california-counties.json")

@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="median_house_value:q"
                    
)

One thing we will try and explore in this notebook is if clustering the houses has any direct relationship with their prices, so we will bucket the houses into intervals of $50000 and re perform the color codes based on each bucket.

In [ ]:
bucketprice = Int.(div.(houses[!,:median_house_value],50000))
insertcols!(houses,3,:cprice=bucketprice)

@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="cprice:n"
                    
)

### 🟤K-means clustering

In [ ]:
X = houses[!, [:latitude,:longitude]]
C = kmeans(Matrix(X)', 10) 
insertcols!(houses,3,:cluster10=>C.assignments)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="cluster10:n"
                    
)

Yes, location affects price of the house but this means location as in proximity to water, prosimity to downtown, promisity to a bus stop and so on

lets' see if this remains true for the rest.

### 🟤K-medoids clustering
For this type of clustering, we need to build a distance matrix. We will use the `Distances` package for this purpose and compute the pairwise Euclidean distances.

In [ ]:
xmatrix = Matrix(X)'
D = pairwise(Euclidean(), xmatrix, xmatrix,dims=2) 

K = kmedoids(D,10)
insertcols!(houses,3,:medoids_clusters=>K.assignments)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="medoids_clusters:n"
                    
)

### 🟤Hierarchial Clustering

In [ ]:
K = hclust(D)
L = cutree(K;k=10)
insertcols!(houses,3,:hclust_clusters=>L)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="hclust_clusters:n"
                    
)

### 🟤DBscan

In [ ]:
?dbscan

In [ ]:
using Distances
dclara = pairwise(SqEuclidean(), Matrix(X)',dims=2)
L = dbscan(dclara, 0.05, 10)
@show length(unique(L.assignments))

In [ ]:
insertcols!(houses,3,:dbscanclusters3=>L.assignments)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
    
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="dbscanclusters3:n"
                    
)

# Finally...
After finishing this notebook, you should be able to:
- [ ] run kmeans clustering on your data
- [ ] run kmedoids clustering on your data
- [ ] run hierarchial clustering on your data
- [ ] run DBscan clustering on your data
- [ ] modify a dataframe and add a new named column
- [ ] generate good looking plots of maps using the VegaLite package

# 🥳 One cool finding

Prices in California do not seem to have an exact mapping with geographical locations. In specifc, performing a clustering algorithm on the houses dataset we had did not reveal a mapping with the price ranges. This indicate that prices relationship to geographical location is not necessairly based on neighborhood but probably other factors like closeness to the water or closeness to a downtown. Here is a figure with a heat map of prices 
<img src="data/0501.png" width="400">
And here is a and k-means clustering of the same houses based on their location
<img src="data/0502.png" width="400">
